# Monte Carlo Reinforcement Learning on Frozen Lake

The following code solves the frozen lake OpenAI Gym environment by using Monte Carlo reinforcement learning. This learns directly from episodes of experience without any prior knowledge of MDP transitions. This code implements the monte carlo first visit with epsilon soft method, which averages the only first visit of the epsisode and uses an epsilon greedy approach to explore. 

In [1]:
# Import Packages
import gym
import numpy as np

## The Environment 
The agent controls the movement of a character in a grid world. Some tiles of the grid are walkable, and others lead to the agent falling into the water. Additionally, the movement direction of the agent is uncertain and only partially depends on the chosen direction. The agent is rewarded for finding a walkable path to a goal tile.

In [2]:
# Create environment
env = gym.make("FrozenLake-v0")

In [3]:
# Get state space and action space size
action_size = env.action_space.n
state_size = env.observation_space.n
print('State size: ', state_size)
print('Action size: ', action_size)

State size:  16
Action size:  4


S = start  
F = frozen, can be walked on  
H = hole, will end the episode  
G = goal, finishing point with reward

In [4]:
s = env.reset()
env.render()


SFFF
FHFH
FFFH
HFFG


In [5]:
# Information on the actions at state s
info = env.P[s]
for key, value in info.items():
    print('Action:',  key,)
    print(value)

Action: 0
[(0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 4, 0.0, False)]
Action: 1
[(0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 4, 0.0, False), (0.3333333333333333, 1, 0.0, False)]
Action: 2
[(0.3333333333333333, 4, 0.0, False), (0.3333333333333333, 1, 0.0, False), (0.3333333333333333, 0, 0.0, False)]
Action: 3
[(0.3333333333333333, 1, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 0, 0.0, False)]


The environment transition probabilities show that the agent is equally likely to end up in 3 different states following a particular action, meaning the agent is unlikely to act as to desired. 

## Monte Carlo Algorithm
This algorithm involves two steps:  
* Policy evaluation: given a fixed policy (state -> action), return the expected reward for each (state, action) pair following this policy in one episode
* Policy improvement: given the accumulated rewards of each (state, action) pair, update the Q-table by using a running average of Q-value of each (state, action) pair

In [6]:
# Parameters
gamma = 0.9      # discount factor
epsilon = 0.1    # exploration rate

### Policy Evaluation
Evaluates a fixed policy with a degree of exploration.   
sar_list = [(state, action, reward), ....]  
sag_list = [(state, action, expected reward), ...]

In [7]:
def policy_evaluation(policy):
    '''
    Function to evaluate a policy and outputs
    a dictionary that provides the expected 
    discounted rewards from that state, taking 
    that action
    
    input: policy[state] -> action
    output: sag_list[state, action] -> expected discounted rewards 
    '''
    
    # Initialise the return list
    G = 0
    sag_list = []
    
    # loop until an episode with valid reward is found i.e until the goal has been reached by acting randomly
    while not sag_list:
    
        # Initialise episode
        state = env.reset()
        action = policy[state]
        sar_list = [(state, action, 0)]
        
        done = False 
        reward_sum = 0
        step = 0
        
        while not done:
            
            # take an action
            next_state, reward, done, info = env.step(action)
            
            # Append state, action, reward to visitied list
            if done:
                
                # Increase the final reward if goal is reached and negative reward if fall into a hole  
                reward = reward*10 if reward !=0 else -1
                
                # append state, action, reward
                sar_list.append((next_state, None, reward))
            else:
                
                # epsilon greedy action
                action = policy[next_state] if np.random.uniform(0,1) > epsilon else env.action_space.sample()
                
                # append state, action, reward
                sar_list.append((next_state, action, reward))
               
            # Cumulating values
            reward_sum += reward
            step += 1
        
        # Backpropagate rewards only if last reward exists
        if sar_list[-1][-1]:
            for state, action, reward in sar_list[::-1]:
                G = reward + gamma * G
                sag_list.append((state, action, G))
                sag_list.reverse()
                
    return sag_list

### Policy Improvement
Following a policy evaluation and a cumulated reward table was obtained for (state, action) pair, the Q-table can be updated with new averages of Q-value. First visit monte carlo averaging was applied here. At the end, the policy is updated using the new values in the Q-table. 

In [8]:
# Initialize training 
Q_table = np.zeros((state_size, action_size))
N_visit = np.zeros((state_size, action_size))
policy = np.random.randint(0,action_size, size=(state_size))
N = 5000

# loop for N episodes
for _ in range(0, N):
    
    # Initialise for each episode
    sag_list = policy_evaluation(policy)
    visited = set()
    
    # Policy improvement loop
    for state, action, G in sag_list:
        
        # Add to the visited list (MC first visit)
        if (state, action) not in visited:
            N_visit[state][action] += 1
            visited.add((state, action))
            
            # Update q-table value with the largest G for that (state, action) pair
            Q_table[state][action] = Q_table[state][action] + 1/N_visit[state][action]*(G - Q_table[state][action])
    
    # Update policy table
    for s in range(0, state_size):
        policy[s] = np.argmax(Q_table[s])

## Policy Testing

In [9]:
def run_game(policy):
    '''
    Function to run one game
    input: policy table
    return: sum of reward 
    '''
    
    # initialise
    state = env.reset()
    done = False 
    reward_sum = 0
    
    # until done 
    while not done:
        
        # take an action in the max q_table
        action = policy[state]
        state, reward, done, info = env.step(action)
        
        # accumulate rewards
        reward_sum += reward
        
    return reward_sum  

In [10]:
def test_policy(policy, r):
    '''
    Function to test a policy r times
    input: a policy table, and r repeated times
    return: percentage of wins 
    '''
    
    # Initialise win
    wins = 0
    
    # loop for r times
    for i in range(r):
        
        # run games 
        reward = run_game(policy)
        
        # accumulate winning games
        if reward == 1:
            wins += 1
                
    return wins / r

In [20]:
print("Out of {} games played, the winning percentage is {}%".format(1000, test_policy(policy, 1000)*100))

Out of 1000 games played, the winning percentage is 43.9%
